In [14]:
import pandas as pd
import numpy as np
import sqlite3
import zipfile
import os
import time
pd.options.display.max_columns = None

# Load data

In [15]:
if os.path.exists("../00_Data/database.sqlite") == False:
    print("Extracting sqlite database from zip file")
    with zipfile.ZipFile("../00_Data/archive.zip","r") as zip_ref:
        zip_ref.extractall("../00_Data/")

In [16]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("../00_Data/database.sqlite")

league = pd.read_sql_query("SELECT * FROM League", con) 
team = pd.read_sql_query("SELECT * FROM Team;", con)
match = pd.read_sql_query("SELECT * FROM Match", con)
Team_Attr = pd.read_sql_query("SELECT * FROM Team_Attributes;", con)

con.close()

# Data Pre-Processing

In [17]:
EPL_ID = list(league[league['name'] == 'England Premier League']['id'])[0]
matches = match[match['league_id'] == EPL_ID].copy()
matches = matches.rename(columns={"id": "match_ID"})
matches = matches.iloc[:,0:11]

teams = team[['team_api_id', 'team_long_name']]

In [18]:
# Replace Team ID's with Team Names
matches = pd.merge(matches, teams, how='left', left_on=['home_team_api_id'], right_on=['team_api_id'], indicator=True)
matches = matches.drop(['home_team_api_id', 'team_api_id'], axis=1)
matches = matches.rename(columns={'team_long_name': 'home_team'})
matches = matches.rename(columns={'_merge': '_merge_home'})

matches = pd.merge(matches, teams, how='left', left_on=['away_team_api_id'], right_on=['team_api_id'], indicator=True)
matches = matches.drop(['away_team_api_id', 'team_api_id'], axis=1)
matches = matches.rename(columns={'team_long_name': 'away_team'})
matches = matches.rename(columns={'_merge': '_merge_away'})

In [19]:
Team_Attributes = pd.merge(Team_Attr, teams, how='left', on=['team_api_id'], indicator=True)
Team_Attributes = Team_Attributes.drop(['team_api_id', 'team_fifa_api_id'], axis=1)
Team_Attributes = Team_Attributes.rename(columns={'team_long_name':'team_name'})
Team_Attributes = Team_Attributes.rename(columns={'_merge': '_merge_team_attr'})
Team_Attributes = Team_Attributes.drop('buildUpPlayDribbling', axis=1)

In [20]:
list(matches['home_team'].unique())

['Manchester United',
 'Arsenal',
 'Sunderland',
 'West Ham United',
 'Aston Villa',
 'Everton',
 'Middlesbrough',
 'Bolton Wanderers',
 'Hull City',
 'Chelsea',
 'Stoke City',
 'Liverpool',
 'Newcastle United',
 'Fulham',
 'Tottenham Hotspur',
 'West Bromwich Albion',
 'Portsmouth',
 'Manchester City',
 'Blackburn Rovers',
 'Wigan Athletic',
 'Wolverhampton Wanderers',
 'Burnley',
 'Birmingham City',
 'Blackpool',
 'Queens Park Rangers',
 'Norwich City',
 'Swansea City',
 'Reading',
 'Southampton',
 'Crystal Palace',
 'Cardiff City',
 'Leicester City',
 'Bournemouth',
 'Watford']

In [21]:
print(Team_Attributes.shape)
Team_Attributes = Team_Attributes[Team_Attributes['team_name'].isin(list(matches['home_team'].unique()))]
print(Team_Attributes.shape)

(1458, 24)
(204, 24)


In [22]:
Team_Attributes.team_name.value_counts()

Liverpool                  6
Swansea City               6
Bolton Wanderers           6
Leicester City             6
Queens Park Rangers        6
Bournemouth                6
Birmingham City            6
Fulham                     6
Arsenal                    6
Stoke City                 6
Wolverhampton Wanderers    6
Aston Villa                6
Newcastle United           6
Middlesbrough              6
Wigan Athletic             6
Southampton                6
Sunderland                 6
Blackpool                  6
Tottenham Hotspur          6
Blackburn Rovers           6
Manchester City            6
Cardiff City               6
Burnley                    6
Portsmouth                 6
Norwich City               6
Watford                    6
Chelsea                    6
Reading                    6
Everton                    6
West Ham United            6
West Bromwich Albion       6
Hull City                  6
Manchester United          6
Crystal Palace             6
Name: team_nam

In [23]:
Team_Attributes.date.unique()

array(['2010-02-22 00:00:00', '2011-02-22 00:00:00',
       '2012-02-22 00:00:00', '2013-09-20 00:00:00',
       '2014-09-19 00:00:00', '2015-09-10 00:00:00'], dtype=object)

In [24]:
Team_Attributes.loc[Team_Attributes['date'] == '2010-02-22 00:00:00', 'season'] = '2009/2010'
Team_Attributes.loc[Team_Attributes['date'] == '2011-02-22 00:00:00', 'season'] = '2010/2011'
Team_Attributes.loc[Team_Attributes['date'] == '2012-02-22 00:00:00', 'season'] = '2011/2012'
Team_Attributes.loc[Team_Attributes['date'] == '2013-09-20 00:00:00', 'season'] = '2012/2013'
Team_Attributes.loc[Team_Attributes['date'] == '2014-09-19 00:00:00', 'season'] = '2013/2014'
Team_Attributes.loc[Team_Attributes['date'] == '2015-09-10 00:00:00', 'season'] = '2014/2015'

Team_Attributes = Team_Attributes.rename(columns={'team_name':'home_team'})

In [25]:
Team_Attributes.head()

,id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,buildUpPlayPositioningClass,chanceCreationPassing,chanceCreationPassingClass,chanceCreationCrossing,chanceCreationCrossingClass,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass,home_team,_merge_team_attr,season
70,71,2010-02-22 00:00:00,66,Balanced,Little,30,Short,Free Form,30,Safe,45,Normal,35,Normal,Free Form,30,Deep,40,Press,50,Normal,Cover,Arsenal,both,2009/2010
71,72,2011-02-22 00:00:00,75,Fast,Little,40,Mixed,Free Form,40,Normal,45,Normal,65,Normal,Free Form,50,Medium,40,Press,45,Normal,Cover,Arsenal,both,2010/2011
72,73,2012-02-22 00:00:00,25,Slow,Little,32,Short,Organised,41,Normal,34,Normal,30,Little,Free Form,57,Medium,57,Press,52,Normal,Cover,Arsenal,both,2011/2012
73,74,2013-09-20 00:00:00,30,Slow,Little,29,Short,Organised,41,Normal,55,Normal,39,Normal,Free Form,64,Medium,54,Press,52,Normal,Cover,Arsenal,both,2012/2013
74,75,2014-09-19 00:00:00,59,Balanced,Normal,26,Short,Organised,28,Safe,55,Normal,64,Normal,Free Form,51,Medium,44,Press,52,Normal,Cover,Arsenal,both,2013/2014


# Add Win Percentage Column

In [26]:
matches['home_result'] = None
matches.loc[matches['home_team_goal'] > matches['away_team_goal'], 'home_result'] = 'W'
matches.loc[matches['home_team_goal'] < matches['away_team_goal'], 'home_result'] = 'L'
matches.loc[matches['home_team_goal'] == matches['away_team_goal'], 'home_result'] = 'D'

In [27]:
home_win_perc = matches[(matches['home_result'] == 'W')].groupby(['season','home_team', 
                    'home_result'])['match_ID'].count()

home_draw_perc = matches[(matches['home_result'] == 'D')].groupby(['season','home_team', 
                    'home_result'])['match_ID'].count()

away_win_perc = matches[(matches['home_result'] == 'L')].groupby(['season','away_team', 
                    'home_result'])['match_ID'].count()

away_draw_perc = matches[(matches['home_result'] == 'D')].groupby(['season','away_team', 
                    'home_result'])['match_ID'].count()

# Create final Dataset

In [28]:
dataset = pd.merge(Team_Attributes, home_win_perc, how='inner', on=['home_team', 'season'], indicator=True)
dataset = dataset.rename(columns={'match_ID':'home_team_win_perc'})
dataset = dataset.rename(columns={'_merge': '_merge_home_win_perc'})
dataset = pd.merge(dataset, away_win_perc, how='inner', left_on=['home_team', 'season'],
                   right_on=['away_team', 'season'], indicator=True)
dataset = dataset.rename(columns={'match_ID':'away_team_win_perc'})
dataset = dataset.rename(columns={'_merge': '_merge_away_win_perc'})
dataset = dataset.drop(['_merge_team_attr', '_merge_home_win_perc', '_merge_away_win_perc'], axis=1)
dataset = dataset.rename(columns={'home_team':'team'})

In [29]:
dataset = pd.merge(dataset, home_draw_perc, how='left', left_on=['team', 'season'],
                   right_on=['home_team', 'season'], indicator=True)
dataset = dataset.rename(columns={'_merge': '_merge_home_draw_perc'})
dataset = dataset.rename(columns={'match_ID':'home_team_draw_perc'})

dataset = pd.merge(dataset, away_draw_perc, how='left', left_on=['team', 'season'],
                   right_on=['away_team', 'season'], indicator=True)
dataset = dataset.rename(columns={'_merge': '_merge_away_draw_perc'})
dataset = dataset.rename(columns={'match_ID':'away_team_draw_perc'})
dataset = dataset.drop(['_merge_home_draw_perc','_merge_away_draw_perc'],axis=1)

#### If a team has not drawn a game in the entire season: impute the perc to zero.

In [30]:
dataset.loc[dataset['home_team_draw_perc'].isna(), 'home_team_draw_perc'] = 0
dataset.loc[dataset['away_team_draw_perc'].isna(), 'away_team_draw_perc'] = 0
dataset['home_team_draw_perc'] = dataset['home_team_draw_perc'].astype(int)

### Get overall win percentage

In [31]:
# Get overall win_perc
dataset['step1'] = (dataset['home_team_win_perc'] + dataset['away_team_win_perc'])
dataset['step2'] = (dataset['home_team_draw_perc'] + dataset['away_team_draw_perc'])
dataset['overall_win_perc'] = (dataset['step1']*(3/4) + dataset['step2']*(1/4)) / 38

In [32]:
dataset = dataset.rename(columns={'home_team_win_perc':'home_team_win', 'away_team_win_perc':'away_team_win', 'home_team_draw_perc':'home_team_draw', 'away_team_draw_perc':'away_team_draw', 'step1':'total_win', 'step2':'total_draw'})
dataset.head()

,id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,buildUpPlayPositioningClass,chanceCreationPassing,chanceCreationPassingClass,chanceCreationCrossing,chanceCreationCrossingClass,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass,team,season,home_team_win,away_team_win,home_team_draw,away_team_draw,total_win,total_draw,overall_win_perc
0,71,2010-02-22 00:00:00,66,Balanced,Little,30,Short,Free Form,30,Safe,45,Normal,35,Normal,Free Form,30,Deep,40,Press,50,Normal,Cover,Arsenal,2009/2010,15,8,2,4,23,6,0.493421
1,72,2011-02-22 00:00:00,75,Fast,Little,40,Mixed,Free Form,40,Normal,45,Normal,65,Normal,Free Form,50,Medium,40,Press,45,Normal,Cover,Arsenal,2010/2011,11,8,4,7,19,11,0.447368
2,73,2012-02-22 00:00:00,25,Slow,Little,32,Short,Organised,41,Normal,34,Normal,30,Little,Free Form,57,Medium,57,Press,52,Normal,Cover,Arsenal,2011/2012,12,9,4,3,21,7,0.460526
3,74,2013-09-20 00:00:00,30,Slow,Little,29,Short,Organised,41,Normal,55,Normal,39,Normal,Free Form,64,Medium,54,Press,52,Normal,Cover,Arsenal,2012/2013,11,10,5,5,21,10,0.480263
4,75,2014-09-19 00:00:00,59,Balanced,Normal,26,Short,Organised,28,Safe,55,Normal,64,Normal,Free Form,51,Medium,44,Press,52,Normal,Cover,Arsenal,2013/2014,13,11,5,2,24,7,0.519737


# Check the shape of final dataframe, and by each season

In [33]:
dataset.shape

(119, 31)

In [34]:
dataset[dataset['season'] == '2009/2010'].shape

(19, 31)

In [35]:
dataset[dataset['season'] == '2010/2011'].shape

(20, 31)

In [36]:
dataset[dataset['season'] == '2011/2012'].shape

(20, 31)

In [37]:
dataset[dataset['season'] == '2012/2013'].shape

(20, 31)

In [38]:
dataset[dataset['season'] == '2013/2014'].shape

(20, 31)

In [39]:
dataset[dataset['season'] == '2014/2015'].shape

(20, 31)

# Pick teams that appear at least in 4 seasons

In [40]:
s_09_10 = set(list(dataset[dataset['season'] == '2009/2010'].team.unique()))
s_10_11 = set(list(dataset[dataset['season'] == '2010/2011'].team.unique()))
s_11_12 = set(list(dataset[dataset['season'] == '2011/2012'].team.unique()))
s_12_13 = set(list(dataset[dataset['season'] == '2012/2013'].team.unique()))
s_13_14 = set(list(dataset[dataset['season'] == '2013/2014'].team.unique()))
s_14_15 = set(list(dataset[dataset['season'] == '2014/2015'].team.unique()))

team_list = [s_09_10, s_10_11, s_11_12, s_12_13, s_13_14, s_14_15]

In [41]:
a= team_list[0]
for i in range(1,len(team_list)):
    b = team_list[i]
    a =  a.union(b)

all_teams = list(a)

In [42]:
all_teams
all_teams_dict = {}
for team in all_teams:
    all_teams_dict[team] = 0

all_teams_keys = list(all_teams_dict.keys())
for season in team_list:
    for team in season:
        if team in all_teams_keys:
            all_teams_dict[team] = all_teams_dict[team] + 1 

In [43]:
all_teams_dict

{'Crystal Palace': 2,
 'Wolverhampton Wanderers': 3,
 'Wigan Athletic': 4,
 'Fulham': 5,
 'Bolton Wanderers': 3,
 'Tottenham Hotspur': 6,
 'Manchester City': 6,
 'Cardiff City': 1,
 'Chelsea': 6,
 'Hull City': 2,
 'Aston Villa': 6,
 'Manchester United': 6,
 'Stoke City': 6,
 'Liverpool': 6,
 'Newcastle United': 5,
 'Arsenal': 6,
 'Birmingham City': 2,
 'Queens Park Rangers': 3,
 'Leicester City': 1,
 'Blackpool': 1,
 'Blackburn Rovers': 3,
 'Sunderland': 6,
 'Southampton': 3,
 'Burnley': 2,
 'Portsmouth': 1,
 'Norwich City': 3,
 'Swansea City': 4,
 'Reading': 1,
 'Everton': 6,
 'West Ham United': 5,
 'West Bromwich Albion': 5}

In [44]:
final_team_list = []
for key, value in all_teams_dict.items():
    if value >= 4:
        print(key, ': ', value)
        final_team_list.append(key)

Wigan Athletic :  4
Fulham :  5
Tottenham Hotspur :  6
Manchester City :  6
Chelsea :  6
Aston Villa :  6
Manchester United :  6
Stoke City :  6
Liverpool :  6
Newcastle United :  5
Arsenal :  6
Sunderland :  6
Swansea City :  4
Everton :  6
West Ham United :  5
West Bromwich Albion :  5


In [45]:
final_team_list

['Wigan Athletic',
 'Fulham',
 'Tottenham Hotspur',
 'Manchester City',
 'Chelsea',
 'Aston Villa',
 'Manchester United',
 'Stoke City',
 'Liverpool',
 'Newcastle United',
 'Arsenal',
 'Sunderland',
 'Swansea City',
 'Everton',
 'West Ham United',
 'West Bromwich Albion']

# Subset data to the identified teams

In [46]:
subset_dataset = dataset[dataset['team'].isin(final_team_list)]

In [47]:
subset_dataset.team.value_counts()

Stoke City              6
Manchester United       6
Aston Villa             6
Everton                 6
Chelsea                 6
Arsenal                 6
Tottenham Hotspur       6
Liverpool               6
Sunderland              6
Manchester City         6
West Bromwich Albion    5
Fulham                  5
West Ham United         5
Newcastle United        5
Wigan Athletic          4
Swansea City            4
Name: team, dtype: int64

In [48]:
subset_dataset.shape

(88, 31)

# Create final dataset csv file

In [49]:
subset_dataset.to_csv('../00_Data/final_dataset.csv')

# Create mock up data for 2016 season

In [50]:
mockup_data = dataset[dataset['season'] == '2014/2015'].copy()

In [51]:
mockup_data['season'] = '2015/2016'
team_list = ['Leicester City','Arsenal','Tottenham Hotspur','Manchester City','Manchester United','Southampton','West Ham United','Liverpool','Stoke City','Chelsea','Everton','Swansea City','Watford','West Bromwich Albion','Crystal Palace','Bournemouth','Sunderland','Newcastle United','Norwich City','Aston Villa']
mockup_data = mockup_data.rename(columns={'team':'home_team'})
mockup_data = mockup_data.drop(['home_team_win', 'away_team_win','home_team_draw','away_team_draw','total_win','total_draw','overall_win_perc'], axis=1)
mockup_data['home_team'] = team_list

##### Append win, draw and loss

In [52]:
mockup_data = pd.merge(mockup_data, home_win_perc, how='inner', on=['home_team', 'season'], indicator=True)
mockup_data = mockup_data.rename(columns={'match_ID':'home_team_win_perc'})
mockup_data = mockup_data.rename(columns={'_merge': '_merge_home_win_perc'})
mockup_data = pd.merge(mockup_data, away_win_perc, how='inner', left_on=['home_team', 'season'],
                   right_on=['away_team', 'season'], indicator=True)
mockup_data = mockup_data.rename(columns={'match_ID':'away_team_win_perc'})
mockup_data = mockup_data.rename(columns={'_merge': '_merge_away_win_perc'})
mockup_data = mockup_data.drop(['_merge_home_win_perc', '_merge_away_win_perc'], axis=1)
mockup_data = mockup_data.rename(columns={'home_team':'team'})

In [53]:
mockup_data = pd.merge(mockup_data, home_draw_perc, how='left', left_on=['team', 'season'],
                   right_on=['home_team', 'season'], indicator=True)
mockup_data = mockup_data.rename(columns={'_merge': '_merge_home_draw_perc'})
mockup_data = mockup_data.rename(columns={'match_ID':'home_team_draw_perc'})

mockup_data = pd.merge(mockup_data, away_draw_perc, how='left', left_on=['team', 'season'],
                   right_on=['away_team', 'season'], indicator=True)
mockup_data = mockup_data.rename(columns={'_merge': '_merge_away_draw_perc'})
mockup_data = mockup_data.rename(columns={'match_ID':'away_team_draw_perc'})
mockup_data = mockup_data.drop(['_merge_home_draw_perc','_merge_away_draw_perc'],axis=1)

If a team has not drawn an entire season, impute the value to 0

In [54]:
mockup_data.loc[mockup_data['home_team_draw_perc'].isna(), 'home_team_draw_perc'] = 0
mockup_data.loc[mockup_data['away_team_draw_perc'].isna(), 'away_team_draw_perc'] = 0
mockup_data['home_team_draw_perc'] = mockup_data['home_team_draw_perc'].astype(int)

In [55]:
# Get overall win_perc
mockup_data['step1'] = (mockup_data['home_team_win_perc'] + mockup_data['away_team_win_perc'])
mockup_data['step2'] = (mockup_data['home_team_draw_perc'] + mockup_data['away_team_draw_perc'])
mockup_data['overall_win_perc'] = (mockup_data['step1']*(3/4) + mockup_data['step2']*(1/4)) / 38

In [56]:
mockup_data = mockup_data.rename(columns={'home_team_win_perc':'home_team_win', 'away_team_win_perc':'away_team_win', 'home_team_draw_perc':'home_team_draw', 'away_team_draw_perc':'away_team_draw', 'step1':'total_win', 'step2':'total_draw'})
mockup_data.head()

,id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,buildUpPlayPositioningClass,chanceCreationPassing,chanceCreationPassingClass,chanceCreationCrossing,chanceCreationCrossingClass,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass,team,season,home_team_win,away_team_win,home_team_draw,away_team_draw,total_win,total_draw,overall_win_perc
0,76,2015-09-10 00:00:00,59,Balanced,Normal,30,Short,Organised,28,Safe,44,Normal,46,Normal,Free Form,51,Medium,44,Press,52,Normal,Cover,Leicester City,2015/2016,12,11,6,6,23,12,0.532895
1,82,2015-09-10 00:00:00,63,Balanced,Normal,54,Mixed,Organised,60,Normal,48,Normal,38,Normal,Organised,35,Medium,44,Press,54,Normal,Cover,Arsenal,2015/2016,12,8,4,7,20,11,0.467105
2,254,2015-09-10 00:00:00,54,Balanced,Little,67,Long,Organised,72,Risky,57,Normal,47,Normal,Organised,63,Medium,62,Press,58,Normal,Cover,Tottenham Hotspur,2015/2016,10,9,6,7,19,13,0.460526
3,312,2015-09-10 00:00:00,67,Fast,Normal,36,Mixed,Organised,41,Normal,34,Normal,44,Normal,Organised,39,Medium,41,Press,46,Normal,Cover,Manchester City,2015/2016,12,7,2,7,19,9,0.434211
4,341,2015-09-10 00:00:00,61,Balanced,Normal,67,Long,Organised,67,Risky,69,Lots,50,Normal,Organised,36,Medium,34,Press,41,Normal,Cover,Manchester United,2015/2016,12,7,5,4,19,9,0.434211


In [57]:
mockup_data.shape

(20, 31)

In [58]:
mockup_data.defenceDefenderLineClass.value_counts()

Cover    20
Name: defenceDefenderLineClass, dtype: int64

In [59]:
buildUpPlaySpeedClass = ['Fast', 'Fast', 'Fast', 'Fast', 'Fast', 'Balanced', 'Balanced', 'Balanced', 'Balanced', 'Balanced', 'Balanced', 'Balanced', 'Balanced', 'Slow', 'Slow', 'Slow', 'Slow', 'Slow', 'Slow', 'Slow']
buildUpPlayDribblingClass = ['Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Little', 'Normal', 'Little', 'Little', 'Little', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Little']
buildUpPlayPassingClass = ['Short', 'Short', 'Mixed', 'Mixed', 'Mixed', 'Mixed', 'Mixed', 'Mixed', 'Mixed', 'Long', 'Mixed', 'Long', 'Mixed', 'Mixed', 'Mixed', 'Mixed', 'Mixed', 'Mixed', 'Mixed', 'Long']
buildUpPlayPositioningClass = ['Free Form', 'Organised', 'Organised', 'Organised', 'Organised', 'Organised', 'Organised', 'Organised', 'Organised', 'Organised', 'Organised', 'Organised', 'Organised', 'Organised', 'Organised', 'Organised', 'Organised', 'Organised', 'Organised', 'Organised']
chanceCreationPassingClass = ['Normal', 'Risky', 'Normal', 'Normal', 'Risky', 'Risky', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal']

chanceCreationCrossingClass = ['Lots', 'Lots', 'Lots', 'Normal', 'Little', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Little', 'Little']
chanceCreationShootingClass = ['Normal', 'Normal', 'Normal', 'Normal', 'Little', 'Little', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Little', 'Little']
chanceCreationPositioningClass = ['Free Form', 'Free Form', 'Free Form', 'Free Form', 'Organised', 'Organised', 'Organised', 'Organised', 'Organised', 'Organised', 'Organised', 'Organised', 'Organised', 'Organised', 'Organised', 'Organised', 'Organised', 'Organised', 'Organised', 'Organised']
defencePressureClass = ['Medium', 'Medium', 'Medium', 'Medium', 'Deep', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Deep', 'Deep', 'Deep', 'Deep']

mockup_data['buildUpPlaySpeedClass'] = buildUpPlaySpeedClass
mockup_data['buildUpPlayDribblingClass'] = buildUpPlayDribblingClass
mockup_data['buildUpPlayPassingClass'] = buildUpPlayPassingClass
mockup_data['buildUpPlayPositioningClass'] = buildUpPlayPositioningClass
mockup_data['chanceCreationPassingClass'] = chanceCreationPassingClass

mockup_data['chanceCreationCrossingClass'] = chanceCreationCrossingClass
mockup_data['chanceCreationShootingClass'] = chanceCreationShootingClass
mockup_data['chanceCreationPositioningClass'] = chanceCreationPositioningClass
mockup_data['defencePressureClass'] = defencePressureClass

In [60]:
mockup_data

,id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,buildUpPlayPositioningClass,chanceCreationPassing,chanceCreationPassingClass,chanceCreationCrossing,chanceCreationCrossingClass,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass,team,season,home_team_win,away_team_win,home_team_draw,away_team_draw,total_win,total_draw,overall_win_perc
0,76,2015-09-10 00:00:00,59,Fast,Normal,30,Short,Free Form,28,Normal,44,Lots,46,Normal,Free Form,51,Medium,44,Press,52,Normal,Cover,Leicester City,2015/2016,12,11,6,6,23,12,0.532895
1,82,2015-09-10 00:00:00,63,Fast,Normal,54,Short,Organised,60,Risky,48,Lots,38,Normal,Free Form,35,Medium,44,Press,54,Normal,Cover,Arsenal,2015/2016,12,8,4,7,20,11,0.467105
2,254,2015-09-10 00:00:00,54,Fast,Normal,67,Mixed,Organised,72,Normal,57,Lots,47,Normal,Free Form,63,Medium,62,Press,58,Normal,Cover,Tottenham Hotspur,2015/2016,10,9,6,7,19,13,0.460526
3,312,2015-09-10 00:00:00,67,Fast,Normal,36,Mixed,Organised,41,Normal,34,Normal,44,Normal,Free Form,39,Medium,41,Press,46,Normal,Cover,Manchester City,2015/2016,12,7,2,7,19,9,0.434211
4,341,2015-09-10 00:00:00,61,Fast,Normal,67,Mixed,Organised,67,Risky,69,Little,50,Little,Organised,36,Deep,34,Press,41,Normal,Cover,Manchester United,2015/2016,12,7,5,4,19,9,0.434211
5,410,2015-09-10 00:00:00,43,Balanced,Normal,40,Mixed,Organised,39,Risky,33,Normal,63,Little,Organised,52,Medium,58,Press,59,Normal,Cover,Southampton,2015/2016,11,7,3,6,18,9,0.414474
6,602,2015-09-10 00:00:00,56,Balanced,Normal,55,Mixed,Organised,54,Normal,49,Normal,56,Normal,Organised,38,Medium,37,Press,46,Normal,Cover,West Ham United,2015/2016,9,7,7,7,16,14,0.407895
7,718,2015-09-10 00:00:00,63,Balanced,Normal,60,Mixed,Organised,47,Normal,64,Normal,46,Normal,Organised,58,Medium,65,Press,55,Normal,Cover,Liverpool,2015/2016,8,8,8,4,16,12,0.394737
8,748,2015-09-10 00:00:00,66,Balanced,Little,45,Mixed,Organised,34,Normal,34,Normal,46,Normal,Organised,51,Medium,52,Press,61,Normal,Cover,Stoke City,2015/2016,8,6,4,5,14,9,0.335526
9,806,2015-09-10 00:00:00,59,Balanced,Normal,29,Long,Organised,38,Normal,36,Normal,24,Normal,Organised,48,Medium,47,Press,54,Normal,Cover,Chelsea,2015/2016,5,7,9,5,12,14,0.328947


In [61]:
subset_mockup_data = mockup_data[mockup_data['team'].isin(final_team_list)]
subset_mockup_data.shape

(14, 31)

In [62]:
subset_mockup_data = mockup_data

# Output mockup data into csv

In [63]:
subset_mockup_data.to_csv('../00_Data/mockup_2015-2016_dataset.csv')